In [1]:
ds_path = '/home/boris/Projects/Voice_Assistant_for_Voice_Anomaly_Persons/Multi-lingual Phoneme Recognition/data/xls-r_dataset_v4.csv'

In [22]:
from collections import defaultdict, Counter
from typing import Iterable

import pandas as pd

In [16]:
import json

f = open("/home/boris/Projects/Voice_Assistant_for_Voice_Anomaly_Persons/Multi-lingual Phoneme Recognition/models/processor/vocab.json")
phonemes: dict[str, int] = json.load(f)
f.close()

phonemes_reverse ={j:i for i, j in phonemes.items()}

In [5]:
df = pd.read_csv(ds_path)

In [8]:
df.iloc[0]['IPA'], df.iloc[0]['text']

('k r a n d i', 'радио')

In [11]:
def get_triplets(ipa: list[str], text: str) -> Iterable[tuple[tuple[str, str], str]]:
    for i in range(min(len(ipa)-1, len(text))):
        yield (ipa[i], ipa[i+1]), text[i]

In [31]:
example = get_triplets(df.iloc[0]['IPA'].split(), df.iloc[0]['text'])

In [38]:
def prepare_text(raw: str) -> str:
    return ''.join(i.lower() for i in raw if i.isalpha())

In [39]:
class HMMState:
    _state: defaultdict[tuple[str, str], Counter]
    def __init__(self) -> None:
        self._state = defaultdict(lambda: Counter())

In [40]:
hmm_state = HMMState()

In [41]:
for _, row in df.iterrows():
    for ph, l in get_triplets(row['IPA'].split(), prepare_text(row['text'])):
        hmm_state._state[ph][l]+=1

In [42]:
hmm_state._state

defaultdict(<function __main__.HMMState.__init__.<locals>.<lambda>()>,
            {('k',
              'r'): Counter({'р': 336,
                      'о': 103,
                      'е': 100,
                      'а': 99,
                      'т': 73,
                      'и': 47,
                      'п': 44,
                      'к': 44,
                      'в': 37,
                      'с': 33,
                      'у': 33,
                      'м': 29,
                      'н': 29,
                      'б': 22,
                      'л': 22,
                      'ь': 21,
                      'д': 20,
                      'ы': 19,
                      'я': 19,
                      'г': 16,
                      'й': 12,
                      'ж': 11,
                      'ч': 8,
                      'з': 7,
                      'ш': 6,
                      'ю': 4,
                      'ц': 3,
                      'щ': 3,
                      'х': 2,
        

In [69]:
test_ipa = 't e s t i k r a v a d n e e n a k r o q o b t e k s t e d e l a s k r a v e n e n i m e t k r i k k a tʃ e s t o'.split()

In [70]:
for i in range(len(test_ipa)-1):
    r = hmm_state._state[(test_ipa[i], test_ipa[i+1])].most_common(1)
    if len(r)>0:
        print(r[0][0], end='')

тсстираовонненнаррткбитесстдделссраовннннимткрриркччсст

In [65]:
from Levenshtein import ratio

In [71]:
print(ratio("зотсттуоднддеррсстазнновонеоноскреттеемаркоскеемнттел", 'записьаудиодляраспознаваниескрытыемарковскиемодели'))
print(ratio('тсстираовонненнаррткбитесстдделссраовннннимткрриркччсст', 'записьаудиодляраспознаваниескрытыемарковскиемодели'))

0.5436893203883495
0.34285714285714286


In [72]:
print(ratio("тсстираовонненнаррткбитесстдделссраовннннимткрриркччсст", 'записьаудиодляраспознаваниескрытыемарковскиемодели'))
print(ratio("тсстираовонненнаррткбитесстдделссраовннннимткрриркччсст", "тестированиенадругомтекстедлясравненияметриккачества"))

0.34285714285714286
0.6168224299065421
